# Subsetting the dataset into Test and Train

In [4]:
if(!require(DBI)){install.packages("DBI"); require(DBI)}
if(!require(RSQLite)){install.packages("RSQLite"); require(RSQLite)}
library(dplyr)
library(stringr)
if(!require(imbalance)){install.packages("imbalance");require(imbalance)}
if(!require(ROSE)){install.packages("ROSE");require(ROSE)}

Loading required package: DBI
Loading required package: RSQLite

Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Loading required package: imbalance
Loading required package: ROSE
Loaded ROSE 0.0-3



In [5]:
DATA_PATH = "C:/Users/moshe/Documents/projectML/p1/data/raw/"
DB_FILE = paste0(DATA_PATH,"/hairsalon.db") 
#### Read function to import data from the SQL to a  dataframe.
readSQL <- function(query, db=DB_FILE) {
    require(DBI)
    require(RSQLite)
    con <- dbConnect(SQLite(), DB_FILE)
    df <- dbGetQuery(con, query)
    return(df)
}
writeSQL <- function(df,tablename,db=DB_FILE) {
    require(DBI)
    require(RSQLite)
    con <- dbConnect(SQLite(), DB_FILE)
    found <- dbExistsTable(con, tablename)
    if (found) {
        dbRemoveTable(con, tablename)
    }
    dbWriteTable(con,tablename,df)
}

In [6]:
filepath <- "C:/Users/moshe/Documents/projectML/p1/data/interim/selectedfeatures.csv"
l <- read.csv(file=filepath,header = T)
l <- paste(l[['Variable']],collapse = "\",\"")
l <-paste("\"noshowflag",l,sep="\",\"",collapse="")
selectedfeatures <- paste(l,"\"",sep="")
selectedfeatures

[1] "\"noshowflag\",\"firstapp\",\"multipleappt\",\"previousnoshow\",\"svcavg\",\"lastsvcavg\",\"x0_wkday_1.0_hour_10.0\",\"x0_wkday_1.0_hour_8.0\",\"x0_wkday_3.0_hour_14.0\",\"x0_wkday_4.0_hour_10.0\",\"x0_wkday_4.0_hour_9.0\",\"x0_wkday_5.0_hour_14.0\",\"x1_COLOR\",\"x1_STYLE\",\"x2_CBAL\",\"x2_CDPB\",\"x2_CHLPL\",\"x2_CON\",\"x2_FRI\",\"x2_SBD\",\"x2_SHCW\",\"x3_BECKY\",\"x3_KELLY\",\"x3_SINEAD\",\"x4_JJ\",\"x4_KELLY\",\"x5_lastappt31-45\",\"x7_last60dlatecancel-1\",\"x8_last60regcancel-0\",\"x9_10.0\",\"x9_12.0\",\"x10_5.0\",\"x10_6.0\",\"x11_2.0\",\"x11_3.0\",\"x13_1.0\",\"x13_5.0\",\"last60dbillamt01\",\"last60dcolor01\",\"last60dproduct01\""

In [7]:
query <-  paste('SELECT ',selectedfeatures,' FROM finalbooking')
query
data = readSQL(query)
head(data)
#typeof(data)

[1] "SELECT  \"noshowflag\",\"firstapp\",\"multipleappt\",\"previousnoshow\",\"svcavg\",\"lastsvcavg\",\"x0_wkday_1.0_hour_10.0\",\"x0_wkday_1.0_hour_8.0\",\"x0_wkday_3.0_hour_14.0\",\"x0_wkday_4.0_hour_10.0\",\"x0_wkday_4.0_hour_9.0\",\"x0_wkday_5.0_hour_14.0\",\"x1_COLOR\",\"x1_STYLE\",\"x2_CBAL\",\"x2_CDPB\",\"x2_CHLPL\",\"x2_CON\",\"x2_FRI\",\"x2_SBD\",\"x2_SHCW\",\"x3_BECKY\",\"x3_KELLY\",\"x3_SINEAD\",\"x4_JJ\",\"x4_KELLY\",\"x5_lastappt31-45\",\"x7_last60dlatecancel-1\",\"x8_last60regcancel-0\",\"x9_10.0\",\"x9_12.0\",\"x10_5.0\",\"x10_6.0\",\"x11_2.0\",\"x11_3.0\",\"x13_1.0\",\"x13_5.0\",\"last60dbillamt01\",\"last60dcolor01\",\"last60dproduct01\"  FROM finalbooking"

noshowflag,firstapp,multipleappt,previousnoshow,svcavg,lastsvcavg,x0_wkday_1.0_hour_10.0,x0_wkday_1.0_hour_8.0,x0_wkday_3.0_hour_14.0,x0_wkday_4.0_hour_10.0,...,x9_12.0,x10_5.0,x10_6.0,x11_2.0,x11_3.0,x13_1.0,x13_5.0,last60dbillamt01,last60dcolor01,last60dproduct01
1,1,0,0,-19.02128,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,44.71925,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,1,0,0,77.45883,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
0,1,0,0,77.45883,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,1,0,0,44.71925,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,1,1,0,44.71925,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [8]:
cols.to.factor <- sapply( data, function(col) length(unique(col)) < 10 )
data[ cols.to.factor] <- lapply(data[cols.to.factor] , factor)                        
names(data)

[1] "noshowflag"             "firstapp"               "multipleappt"          
 [4] "previousnoshow"         "svcavg"                 "lastsvcavg"            
 [7] "x0_wkday_1.0_hour_10.0" "x0_wkday_1.0_hour_8.0"  "x0_wkday_3.0_hour_14.0"
[10] "x0_wkday_4.0_hour_10.0" "x0_wkday_4.0_hour_9.0"  "x0_wkday_5.0_hour_14.0"
[13] "x1_COLOR"               "x1_STYLE"               "x2_CBAL"               
[16] "x2_CDPB"                "x2_CHLPL"               "x2_CON"                
[19] "x2_FRI"                 "x2_SBD"                 "x2_SHCW"               
[22] "x3_BECKY"               "x3_KELLY"               "x3_SINEAD"             
[25] "x4_JJ"                  "x4_KELLY"               "x5_lastappt31-45"      
[28] "x7_last60dlatecancel-1" "x8_last60regcancel-0"   "x9_10.0"               
[31] "x9_12.0"                "x10_5.0"                "x10_6.0"               
[34] "x11_2.0"                "x11_3.0"                "x13_1.0"               
[37] "x13_5.0"                "last60dbillamt01"       "last60dcolor01"        
[40] "last60dproduct01"

In [9]:
data <- data %>% rename_at(vars(contains(".")), funs(str_replace_all(., "\\.","_")))
data <- data %>% rename_at(vars(contains("-")), funs(str_replace_all(., "-","_")))
colnames(data)

Warning message:
"funs() is soft deprecated as of dplyr 0.8.0
please use list() instead

# Before:
funs(name = f(.)

# After: 
list(name = ~f(.))
This warning is displayed once per session."

[1] "noshowflag"             "firstapp"               "multipleappt"          
 [4] "previousnoshow"         "svcavg"                 "lastsvcavg"            
 [7] "x0_wkday_1_0_hour_10_0" "x0_wkday_1_0_hour_8_0"  "x0_wkday_3_0_hour_14_0"
[10] "x0_wkday_4_0_hour_10_0" "x0_wkday_4_0_hour_9_0"  "x0_wkday_5_0_hour_14_0"
[13] "x1_COLOR"               "x1_STYLE"               "x2_CBAL"               
[16] "x2_CDPB"                "x2_CHLPL"               "x2_CON"                
[19] "x2_FRI"                 "x2_SBD"                 "x2_SHCW"               
[22] "x3_BECKY"               "x3_KELLY"               "x3_SINEAD"             
[25] "x4_JJ"                  "x4_KELLY"               "x5_lastappt31_45"      
[28] "x7_last60dlatecancel_1" "x8_last60regcancel_0"   "x9_10_0"               
[31] "x9_12_0"                "x10_5_0"                "x10_6_0"               
[34] "x11_2_0"                "x11_3_0"                "x13_1_0"               
[37] "x13_5_0"                "last60dbillamt01"       "last60dcolor01"        
[40] "last60dproduct01"

In [10]:
summary(data)

 noshowflag firstapp multipleappt previousnoshow     svcavg      
 0:1655     0:876    0:1360       0:1619         Min.   :-43.12  
 1: 138     1:917    1: 433       1: 174         1st Qu.: 44.72  
                                                 Median : 63.01  
                                                 Mean   : 65.45  
                                                 3rd Qu.: 77.46  
                                                 Max.   :270.00  
   lastsvcavg     x0_wkday_1_0_hour_10_0 x0_wkday_1_0_hour_8_0
 Min.   :-19.02   0:1737                 0:1786               
 1st Qu.:  0.00   1:  56                 1:   7               
 Median :  0.00                                               
 Mean   : 30.38                                               
 3rd Qu.: 44.72                                               
 Max.   :244.47                                               
 x0_wkday_3_0_hour_14_0 x0_wkday_4_0_hour_10_0 x0_wkday_4_0_hour_9_0
 0:1755                 0:17

In [11]:
if(!require(tableone)){install.packages("tableone");require(tableone)}

Loading required package: tableone


In [12]:
vn <- setdiff(names(data),"noshowflag")

CreateTableOne(data = data,vars = vn,includeNA = T)

                                
                                 Overall      
  n                               1793        
  firstapp = 1 (%)                 917 (51.1) 
  multipleappt = 1 (%)             433 (24.1) 
  previousnoshow = 1 (%)           174 ( 9.7) 
  svcavg (mean (SD))             65.45 (36.71)
  lastsvcavg (mean (SD))         30.38 (40.67)
  x0_wkday_1_0_hour_10_0 = 1 (%)    56 ( 3.1) 
  x0_wkday_1_0_hour_8_0 = 1 (%)      7 ( 0.4) 
  x0_wkday_3_0_hour_14_0 = 1 (%)    38 ( 2.1) 
  x0_wkday_4_0_hour_10_0 = 1 (%)    36 ( 2.0) 
  x0_wkday_4_0_hour_9_0 = 1 (%)     35 ( 2.0) 
  x0_wkday_5_0_hour_14_0 = 1 (%)    48 ( 2.7) 
  x1_COLOR = 1 (%)                 302 (16.8) 
  x1_STYLE = 1 (%)                1423 (79.4) 
  x2_CBAL = 1 (%)                   20 ( 1.1) 
  x2_CDPB = 1 (%)                    9 ( 0.5) 
  x2_CHLPL = 1 (%)                  39 ( 2.2) 
  x2_CON = 1 (%)                    56 ( 3.1) 
  x2_FRI = 1 (%)                     4 ( 0.2) 
  x2_SBD = 1 (%)           

In [13]:
#library(devtools)
#install_github("karpatit/mechkar")
if(!require(mechkar)){install.packages("mechkar"); require(mechkar)}

Loading required package: mechkar
Loading required package: car
Loading required package: carData

Attaching package: 'car'

The following object is masked from 'package:dplyr':

    recode

Loading required package: Hmisc
Loading required package: lattice
Loading required package: survival
Loading required package: Formula
Loading required package: ggplot2

Attaching package: 'Hmisc'

The following objects are masked from 'package:dplyr':

    src, summarize

The following objects are masked from 'package:base':

    format.pval, units

Loading required package: sqldf
Loading required package: gsubfn
Loading required package: proto
Loading required package: pROC
Type 'citation("pROC")' for a citation.

Attaching package: 'pROC'

The following objects are masked from 'package:stats':

    cov, smooth, var

Loading required package: ResourceSelection
ResourceSelection 0.3-4 	 2019-01-08
Loading required package: sjstats

Attaching package: 'sjstats'

The following object is masked from 

In [14]:
Table1(data = data,x = vn,y = "noshowflag")

  |======================================================================| 100%

Coefficient covariances computed by hccm()
Coefficient covariances computed by hccm()
Coefficient covariances computed by hccm()


V1,V2,Pop,0,1,pval
Individuals,n,1793,1655,138,NA
firstapp,1,917 (51.1%),853 (51.5%),64 (46.4%),0.281
multipleappt,1,433 (24.1%),431 (26.0%),2 (1.4%),0
previousnoshow,1,174 (9.7%),150 (9.1%),24 (17.4%),0.002
svcavg,Mean (SD),65.4 (36.7),66.7 (34.7),50.5 (53.4),NA
svcavg,Median (IQR),63.0 (44.7-77.5),70.2 (44.7-77.5),44.7 (21.9-77.5),0.001
lastsvcavg,Mean (SD),30.4 (40.7),30.3 (40.5),30.8 (42.6),NA
lastsvcavg,Median (IQR),0.0 (0.0-44.7),0.0 (0.0-44.7),0.0 (0.0-44.7),0.911
x0_wkday_1_0_hour_10_0,1,56 (3.1%),45 (2.7%),11 (8.0%),0.002
x0_wkday_1_0_hour_8_0,1,7 (0.4%),4 (0.2%),3 (2.2%),0


In [15]:
############### TRAIN_TEST #######################
train_test <- function(data=NULL,train_name=NULL,test_name=NULL,prop=NULL,seed=123,tableone=False)
{
  checkTrainTest <- function(train=NULL,test=NULL) {
     train[["traintest_ind_"]] <- 1
     test[["traintest_ind_"]] <- 2
     data <- rbind(train, test)
     tab <- Table1(data=data, y="traintest_ind_",x=names(train),messages = F)
     vars <- subset(tab, pval < 0.05)$V1
     if (length(vars)==1) {
        message("You got a perfectly balanced training and test datasets")
        message(" ")
     } else {
        message("WARNING: The following variables are not balanced between the training and test datasets:")
       for (v in vars) { message(paste("*",v)) }
          message("You can try to change the seed value until you get a balanced partition.")
          message("Alternatively, you can ommit this warning and exclude those variables from your model")
          message(" ")
       }
       return(tab)
  }
  ## set the seed to make your partition reproductible
  set.seed(seed)
  smp_size <- floor(prop * nrow(data))
  train_ind <- sample(seq_len(nrow(data)), size = smp_size)
  assign(train_name, data[train_ind, ], envir=globalenv())
  assign(test_name, data[-train_ind, ], envir=globalenv())
  tab = checkTrainTest(get(train_name),get(test_name))
  message(paste("Dataset partitioned into:"))
  message(paste(" + Train dataset:", train_name))
  message(paste(" + Test dataset:", test_name))
  if(tableone==TRUE) {
     return(tab)
  } 
}

In [16]:
table(data$noshowflag)
table(data$noshowflag)/nrow(data)*100


   0    1 
1655  138 


        0         1 
92.303402  7.696598 

## resampling original dataset

In [17]:
#resample data
set.seed(42)
rows <- sample(nrow(data))
data <- data[rows, ]
nrow(data)

[1] 1793

## Selecting test from  original dataset

In [18]:
#1000000000
tab1 <- train_test(data = data, train_name = "train", test_name = "test",prop = 0.75, seed = 1000000000, tableone=T)

  |======================================================================| 100%

Coefficient covariances computed by hccm()
Coefficient covariances computed by hccm()
You got a perfectly balanced training and test datasets
 
Dataset partitioned into:
 + Train dataset: train
 + Test dataset: test


In [19]:
size <- paste('train',nrow(train),'test',nrow(test))
size
print('*****train*****')
table(train$noshowflag)
table(train$noshowflag)/nrow(train)*100
print('******************')
print('*****test*****')
table(test$noshowflag)
table(test$noshowflag)/nrow(test)*100
print('******************')

[1] "train 1344 test 449"

[1] "*****train*****"



   0    1 
1244  100 


        0         1 
92.559524  7.440476 

[1] "******************"
[1] "*****test*****"



  0   1 
411  38 


        0         1 
91.536748  8.463252 

[1] "******************"


In [20]:
library(randomForest)

randomForest 4.6-14
Type rfNews() to see new features/changes/bug fixes.

Attaching package: 'randomForest'

The following object is masked from 'package:ggplot2':

    margin

The following object is masked from 'package:dplyr':

    combine



In [21]:
set.seed(10000)
mod1 <- randomForest(noshowflag ~., data=train)
auc1 <- pROC::auc(test$noshowflag, factor(predict(mod1,newdata=test),levels=sort(unique(predict(mod1,newdata=test))),ordered=TRUE))
auc1

Area under the curve: 0.6148

In [22]:
numPositive <- length(which(train$noshowflag == 1))
numNegative <- length(which(train$noshowflag == 0))
nInstances <- numNegative - numPositive
cbind(numPositive=numPositive,numNegative=numNegative,nInstances=nInstances)

numPositive,numNegative,nInstances
100,1244,1144


## oversampling generating

In [23]:
#over sampling
data_balanced_over <- ovun.sample(noshowflag ~ ., data = train, method = "over",seed=100,N=numNegative*2)$data
table(data_balanced_over$noshowflag)
table(data_balanced_over$noshowflag)/nrow(data_balanced_over)*100
set.seed(10000)
mod1 <- randomForest(noshowflag ~., data=data_balanced_over)
auc1 <- pROC::auc(test$noshowflag, factor(predict(mod1,newdata=test),levels=sort(unique(predict(mod1,newdata=test))),ordered=TRUE))
auc1


   0    1 
1244 1244 


 0  1 
50 50 

Area under the curve: 0.673

## undersampling generating

In [24]:
#under sampling
data_balanced_under <- ovun.sample(noshowflag ~ ., data = train, method = "under",seed=100,N=numPositive*2)$data
table(data_balanced_under$noshowflag)
table(data_balanced_under$noshowflag)/nrow(data_balanced_under)*100
set.seed(10000)
mod1 <- randomForest(noshowflag ~., data=data_balanced_under)
auc1 <- pROC::auc(test$noshowflag, factor(predict(mod1,newdata=test),levels=sort(unique(predict(mod1,newdata=test))),ordered=TRUE))
auc1


  0   1 
100 100 


 0  1 
50 50 

Area under the curve: 0.6773

## over under sampling generating

In [25]:
#overunder sampling
totalrow <-round(nrow(train)*1.8)
data_balanced_overunder <- ovun.sample(noshowflag ~ ., data = train, method = "both",seed=100,p=0.5,N=totalrow)$data
table(data_balanced_overunder$noshowflag)
table(data_balanced_overunder$noshowflag)/nrow(data_balanced_overunder)*100
set.seed(10000)
mod1 <- randomForest(noshowflag ~., data=data_balanced_overunder)
auc1 <- pROC::auc(test$noshowflag, factor(predict(mod1,newdata=test),levels=sort(unique(predict(mod1,newdata=test))),ordered=TRUE))
auc1


   0    1 
1195 1224 


       0        1 
49.40058 50.59942 

Area under the curve: 0.704

## training generating a more balanced dataset using ROSE


In [26]:
if(!require(imbalance)){install.packages("imbalance");require(imbalance)}
### ROSE = Random Over-Sampling Examples
if(!require(ROSE)){install.packages("ROSE");require(ROSE)}
totalrow <-round(nrow(train)*1.8)
data.rose <- ROSE(noshowflag ~ ., data = train, p=0.5, N=totalrow, seed = 70000)$data
table(data.rose$noshowflag)
table(data.rose$noshowflag)/nrow(data.rose)*100
set.seed(10000)
mod1 <- randomForest(noshowflag ~., data=data.rose)
auc1 <- pROC::auc(test$noshowflag, factor(predict(mod1,newdata=test),levels=sort(unique(predict(mod1,newdata=test))),ordered=TRUE))
auc1


   0    1 
1222 1197 


       0        1 
50.51674 49.48326 

Area under the curve: 0.7054

### Probability Density Function Estimation Based Oversampling

In [27]:
train2<-data.frame(train)
indx <- sapply(train, is.factor)    
train2[indx] <- lapply( train2[indx], function(x) as.numeric(as.character(x)))
#lapply(train2,class)
newsamples <- pdfos(dataset=train2, numInstances = 1000,classAttr = "noshowflag")

Error in solve.default(covariance) : 
  Lapack routine dgesv: system is exactly singular: U[2,2] = 0


ERROR: Error in pdfos(dataset = train2, numInstances = 1000, classAttr = "noshowflag"): Not a valid method for this dataset. Variance of the positive class is not an invertible matrix


## Train dev dataset obtained from rose 

In [28]:
#100000
tab2 <- train_test(data = data.rose, train_name = "train2", test_name = "dev",prop = 0.8, seed = 100000, tableone=T)

  |======================================================================| 100%

Coefficient covariances computed by hccm()
Coefficient covariances computed by hccm()
Coefficient covariances computed by hccm()
You got a perfectly balanced training and test datasets
 
Dataset partitioned into:
 + Train dataset: train2
 + Test dataset: dev


In [29]:
size <- paste('train',nrow(train2),'dev',nrow(dev))
size
table(train2$noshowflag)
table(train2$noshowflag)/nrow(train2)*100
table(dev$noshowflag)
table(dev$noshowflag)/nrow(dev)*100

[1] "train 1935 dev 484"


  0   1 
981 954 


       0        1 
50.69767 49.30233 


  0   1 
241 243 


       0        1 
49.79339 50.20661 

In [32]:
set.seed(42)
rows <- sample(nrow(data.rose))
data.rose <- data.rose[rows, ]

In [33]:
writeSQL(data.rose,'rfulltrain')
writeSQL(train2,'rtrain') 
writeSQL(dev,'rdev') 
writeSQL(test,'rtest')